# Creates train/test split of raw data

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!sudo apt install openjdk-8-jdk-headless -qq
!pip install pyarrow
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
ERROR: Package 'rsa' requires a different Python: 2.7.17 not in '>=3.5, <4'
openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
The following packages were automatically installed and are no longer

In [2]:
import numpy as np
import pandas as pd
import csv
import pickle
import time
import math
import collections
import os
from tqdm import tqdm
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '35G')
        .set('spark.driver.memory', '35G')
        .set('spark.driver.maxResultSize', '35G'))
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
# arrow enabling is what makes the conversion from pandas to spark dataframe really fast
sc._conf.get('spark.driver.memory')

'35G'

In [4]:
spark

## Read in raw data

In [5]:
dirPath = '/home/ubuntu/Biomed-Data-Science-NLP-Project/Data/'
patientData_filepath = dirPath + 'B220_SAA_v1.csv'
CCSR_filepath = dirPath + 'ICD_to_CCSR_20201_1.csv'

begin = time.time()
# df_pd = pd.read_csv(patientData_filepath, usecols=[0, 1, 2, 3, 5, 6, 7] + list(range(16,41)), nrows=15)

## Uncomment this line if you want to see a simple skinny table example
# df_pd = pd.read_csv(patientData_filepath, usecols=[0, 1, 2, 3, 5, 6, 7], nrows=15)

# df = spark.createDataFrame(df_pd) 
# df_pd.head()


df = spark.read.load(patientData_filepath, format="csv", sep=",", inferSchema="true", header="true")
end = time.time()
print("Reading csv took: {}".format(end - begin))

Reading csv took: 61.42474389076233


In [6]:
begin = time.time()
df = df.select(df.columns[0:8] + df.columns[16:41])
df = df.drop('Type')
end = time.time()
print("Cell took {}".format(end - begin))

Cell took 0.18488240242004395


In [7]:
from pyspark.sql.window import Window
begin = time.time()
my_window = Window.partitionBy().orderBy("ID")

df = df.withColumn("next_Date", lead(df.Date).over(my_window))
df = df.withColumn("next_Id", lead(df.ID).over(my_window))
df = df.withColumn("datediff", datediff(col("next_Date"), col("Date")).alias("datediff"))
end = time.time()
print("Cell took {}".format(end - begin))

Cell took 0.17697405815124512


In [8]:
begin = time.time()
df = df.withColumn("label", when(col("ID") != col("next_Id"), 2)\
              .when(col("datediff") > 30, 0)\
              .when(col("datediff") <= 30, 1)\
              .otherwise(2))
end = time.time()
print("Cell took {}".format(end - begin))
# df.show()

Cell took 0.06490874290466309


check: # union with the test dataset later to not lose these examples???

In [9]:
begin = time.time()
# removes rows that will not be used in training
df = df.filter(col("label") != 2)
# df_label2 = df.filter(col("label") == 2) 

# Drops columns
df = df.drop('next_Id', 'next_Date', 'datediff') 
end = time.time()
print("Cell took {}".format(end - begin))
# df.show()

Cell took 0.04141521453857422


In [10]:
total_count = df.count()
print("Total number of visits: {}".format(total_count))

Total number of visits: 23457733


In [11]:
begin = time.time()
df = df.orderBy(rand(seed=1234))
end = time.time()
print("Cell took {}".format(end - begin))
# df.show()

Cell took 0.02924823760986328


In [12]:
from pyspark.sql.functions import monotonically_increasing_id 
begin = time.time()
df = df.withColumn("index", monotonically_increasing_id())
end = time.time()
print("Cell took {}".format(end - begin))

Cell took 0.021924734115600586


In [13]:
split = int(total_count * 0.8)
print("Total count: {}".format(total_count))
print("Splitting at: {}".format(split))

Total count: 23457733
Splitting at: 18766186


In [14]:
begin = time.time()
train = df.filter(col("index").between(0, split-1)).drop("index") # repartitin or coalesce after filtering -- 1GB per partition rule
# print("Size of train: {}".format(train.count()))
end = time.time()
print("Cell took {}".format(end - begin))

Cell took 0.05051612854003906


In [15]:
begin = time.time()
train.show()
end = time.time()
print("Cell took {}".format(end - begin))

+--------+-----+------+----------+---+---+--------+---------+-------+-------+------+-------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----+
|      ID|Visit|Visits|      Date|Age|Sex|    Race|Dx10_prin| Dx10_1| Dx10_2|Dx10_3| Dx10_4|Dx10_5|Dx10_6|Dx10_7|Dx10_8|Dx10_9|Dx10_10|Dx10_11|Dx10_12|Dx10_13|Dx10_14|Dx10_15|Dx10_16|Dx10_17|Dx10_18|Dx10_19|Dx10_20|Dx10_21|Dx10_22|Dx10_23|Dx10_24|label|
+--------+-----+------+----------+---+---+--------+---------+-------+-------+------+-------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----+
| 9156719|    8|    11|2017-07-02| 26|  M|   Black|   R45851|    F29|   F329|  null|   null|  null|  null|  null|  null|  null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null

In [16]:
begin = time.time()
train_count = train.count()
end = time.time()
print("Cell took {}".format(end - begin), train_count)

Cell took 71.42446827888489 18766186


In [17]:
begin = time.time()
test = df.filter(col("index").between(split,total_count)).drop("index")
end = time.time()
print("Cell took {}".format(end - begin))

Cell took 0.036742448806762695


In [18]:
begin = time.time()
test.show()
end = time.time()
print("Cell took {}".format(end - begin))

+--------+-----+------+----------+---+---+--------+---------+-------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----+
|      ID|Visit|Visits|      Date|Age|Sex|    Race|Dx10_prin| Dx10_1|Dx10_2|Dx10_3|Dx10_4|Dx10_5|Dx10_6|Dx10_7|Dx10_8|Dx10_9|Dx10_10|Dx10_11|Dx10_12|Dx10_13|Dx10_14|Dx10_15|Dx10_16|Dx10_17|Dx10_18|Dx10_19|Dx10_20|Dx10_21|Dx10_22|Dx10_23|Dx10_24|label|
+--------+-----+------+----------+---+---+--------+---------+-------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----+
| 4303535|    1|     6|2015-10-14| 18|  F|Hispanic|     N390|   null|  null|  null|  null|  null|  null|  null|  null|  null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null

In [19]:
begin = time.time()
test_count = test.count()
print(test_count)
end = time.time()
print("Cell took {}".format(end - begin))

4691547
Cell took 69.69048237800598


## Save train and test to csv

In [20]:
# train.write.mode('overwrite').parquet(dirPath + "train")

In [21]:
# train_df = train.toPandas()

In [22]:
begin = time.time()
train.write.format('csv').option('header', True).option('sep',',').mode('overwrite').save(dirPath + 'train')
end = time.time()
print("Cell took {}".format(end - begin))

Cell took 255.01294088363647


In [23]:
begin = time.time()
test.write.format('csv').option('header', True).option('sep',',').mode('overwrite').save(dirPath + 'test')
end = time.time()
print("Cell took {}".format(end - begin))

Cell took 218.47496604919434


In [24]:
begin = time.time()
train_loaded = spark.read.format("csv").load(dirPath + "train",
                     sep=",", inferSchema="true", header="true")
end = time.time()
print("Cell took {}".format(end - begin))
train_count_loaded = train_loaded.count()
print(train_count_loaded, train_count)

Cell took 27.897841930389404
18766186 18766186


In [25]:
begin = time.time()
test_loaded = spark.read.load(dirPath + "test",
                     format="csv", sep=",", inferSchema="true", header="true")
end = time.time()
print("Cell took {}".format(end - begin))
test_count_loaded = test_loaded.count()
print(test_count_loaded, test_count)

Cell took 7.800708055496216
4691547 4691547
